In [1]:
# Q-learning이랑 SARSA를 tabular learning으로 구현
# 47번, 48번 슬라이드 참고

In [2]:
import gym
import random # 탐사하는 데 쓰기
from collections import namedtuple # Q-table 만드는 데 쓸 자료구조
import collections
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 함수 세개 구현
# 1. epsilon-greedy 액션 선택기 구현
# 2. greedy 액션 선택기 구현 (테스트 때)
# 3. 최적 액션 선택기 (1이랑 2에서 쓰임)

In [19]:
def eps_greedy_action(table, state):
  value, action = best_value_and_action(table, state)
  if random.random() < epsilon:
    # 탐사
    return random.randint(0, n_actions-1)
  else:
    # 활용
    return action

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
def greedy_action(table, state):
  value, action = best_value_and_action(table, state)
  return action

In [6]:
def best_value_and_action(table, state):
  best_value = 0
  best_action = 0
  # 지금 가능한 액션 선택지 모두를 테이블에서 돌면서 가장 높은 값과 그 때의 액션 반환
  for action in range(n_actions):
    if table[(state, action)] > best_value:
      best_action = action
      best_value = table[(state, action)] # Q(s, a)
  return best_value, best_action

In [15]:
# 47번 슬라이드
# SARSA 구현
# 알고리즘 내용에서 가장 안쪽 루프의 안 내용
def SARSA(table, state, action, reward, state_next):
  action_next = eps_greedy_action(table, state_next)
  Q_target = reward + GAMMA * table[(state_next, action_next)]
  TD_error = Q_target - table[(state, action)]
  table[(state, action)] = table[(state, action)] + LEARNING_RATE * TD_error
  return

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
# 48번 슬라이드
# Q-learning 구현
# 알고리즘 내용에서 가장 안쪽 루프의 안 내용
def Q_learn(table, state, action, reward, state_next):
  best_value, _ = best_value_and_action(table, state_next)
  Q_target = reward + GAMMA * best_value # max_a(Q(S', a))
  TD_error = Q_target - table[(state, action)]
  table[(state, action)] = table[(state, action)] + LEARNING_RATE * TD_error

  return

In [14]:
# 테스트 함수
# 이전에 model.evaluate
def test(env, table):
  reward_games = [] # 이 리스트에 보상 값 저장해놓고 평균 구할 거임
  for _ in range(TEST_EPISODES):
    state_ = env.reset()
    rewards = 0
    while True:
      next_state_, reward, done, _ = env.step(greedy_action(table, state_))
      state_ = next_state_
      rewards += reward # 이번 게임 누적 합산 보상

      if done:
        reward_games.append(rewards) # 모든 게임들 누적 합산 보상 값이 저장되어 있는 리스트
        break
  return np.mean(reward_games)

In [9]:
seed = 42
random.seed(seed)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# 강화학습 관련 하이퍼파라미터
GAMMA = 0.9 # discount factor
# 환경에 의해 미리 정해진 값일 수도 있음
# 만약 그렇지 않다면 강화학습 모델 개발자의 자유
EPS_DECAY = 0.999 # epsilon decay?
# epsilon 값을 점점 작게
# 왜? 처음에는 탐사에 비중을 더 두고, 나중으로 갈수록 활용의 비중을 높이기 위해
LEARNING_RATE = 0.8 # 슬라이드에서 alpha
MAX_GAMES = 50000 # 5만 에피소드에 걸쳐 학습하기

PRINT_EVERY = 1000 # 1천번 에피소드마다 로그 보기
TEST_EPISODES = 100 # 위의 테스트 함수에서 플레이할 에피소드 갯수

In [20]:
# 환경 소환
env = gym.make("FrozenLake-v1", is_slippery=True)
state_ = env.reset() # 초기화 에피소드 시작하기 직전에
state_length = env.observation_space.n
n_actions = env.action_space.n

reward_cnt = 0
game_cnt = 0 # 여태 몇번의 에피소드를 플레이했는지

# Q-table
table_ = collections.defaultdict(float) # Q 값으로 채워질 테이블

# 테스트에서 얻은 보상들 리스트
test_rewards = []

epsilon = 1.0 # 처음엔 탐사만 하겠다 ---> EPS_DECAY를 여기에 계속 곱해서 서서히 줄어들게 할 거임

while game_cnt < MAX_GAMES:
  action = eps_greedy_action(table_, state_) # behavior rule (Q-learning이든 SARSA든 공통임)
  next_state_, reward, done, _ = env.step(action) # 한번 스텝은 강화학습 사이클 한번
  # next_state_: 다음 상태
  # reward: 보상
  # done: True/False boolean 지금 task가 끝났는지
  # _: info를 반환하도록 구현된 환경들이 있음; 근데 지금 우리는 안 씀

  Q_learn(table_, state_, action, reward, next_state_)
  # SARSA(table_, state_, action, reward, next_state_)
  reward_cnt += reward # 누적 합산 보상
  state_ = next_state_

  # 만약 done이 True면?
  if done:
    epsilon *= EPS_DECAY
    state_ = env.reset()
    reward_cnt = 0
    game_cnt += 1

    if ((game_cnt + 1) % PRINT_EVERY) == 0:
      test_reward = test(env, table_)
      test_rewards.append(test_reward)
      print("현재 {}번째 에피소드 학습 중... {}개 에피소드 평균 보상: {}".format(game_cnt, TEST_EPISODES, test_reward))

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


현재 999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.17
현재 1999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.51
현재 2999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.5
현재 3999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.75
현재 4999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.69
현재 5999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.69
현재 6999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.54
현재 7999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.51
현재 8999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.69
현재 9999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.61
현재 10999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.69
현재 11999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.79
현재 12999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.73
현재 13999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.75
현재 14999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.84
현재 15999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.56
현재 16999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.65
현재 17999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.76
현재 18999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.74
현재 19999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.77
현재 20999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.74
현재 21999번째 에피소드 학습 중... 100개 에피소드 평균 보상: 0.69


KeyboardInterrupt: ignored